# Language Models: Auto-Complete

We will build an auto-complete system.  Auto-complete system is something you may see every day
- When you google something, you often have suggestions to help you complete your search. 
- When you are writing an email, you get suggestions telling you possible endings to your sentence.  



In [ ]:
import math
import random
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.data.path.append('.')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Part 1: Load and Preprocess Data

We will use twitter data.
Load the data and view the first few sentences by running the next cell.

Notice that data is a long string that contains many many tweets.
Observe that there is a line break "\n" between tweets.

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/Datasets/en_US.twitter.txt') as f:
  data=f.read()

print('Data Type:',type(data))
print('Number of letters:',len(data))
print("First 300 letters of the data")
print("-------")
print(data[:300])

print("Last 300 letters of the data")
print("-------")
print(data[-300:])

Data Type: <class 'str'>
Number of letters: 3335477
First 300 letters of the data
-------
How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.
When you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.
they've decided its more fun if I don't.
So Tired D; Played Lazer Tag & Ran A 
Last 300 letters of the data
-------
ust had one a few weeks back....hopefully we will be back soon! wish you the best yo
Colombia is with an 'o'...“: We now ship to 4 countries in South America (fist pump). Please welcome Columbia to the Stunner Family”
#GutsiestMovesYouCanMake Giving a cat a bath.
Coffee after 5 was a TERRIBLE idea.



### Part 1.2 Pre-process the data

Preprocess this data with the following steps:

1. Split data into sentences using "\n" as the delimiter.
1. Split each sentence into tokens. Note that in this assignment we use "token" and "words" interchangeably.
1. Assign sentences into train or test sets.
1. Find tokens that appear at least N times in the training data.
1. Replace tokens that appear less than N times by `<unk>`

In [ ]:
def split_into_sentences(data):
  sentences=data.split('\n')
  sentences=[s.strip() for s in sentences]
  sentences=[s for s in sentences if len(s)>0]

  return sentences

In [ ]:
# test your code
x = """
I have a pen.\nI have an apple. \nAh\nApple pen.\n
"""
print(x)

split_into_sentences(x)


I have a pen.
I have an apple. 
Ah
Apple pen.




['I have a pen.', 'I have an apple.', 'Ah', 'Apple pen.']

The next step is to tokenize sentences (split a sentence into a list of words). 
- Convert all tokens into lower case so that words which are capitalized (for example, at the start of a sentence) in the original text are treated the same as the lowercase versions of the words.
- Append each tokenized list of words into a list of tokenized sentences.

In [ ]:
def tokenize_sentences(sentences):
  tokenized_sentences=[]

  for sentence in sentences:
    sentence=sentence.lower()
    tokenized=nltk.word_tokenize(sentence)
    tokenized_sentences.append(tokenized)
  return tokenized_sentences



In [ ]:
# test your code
sentences = ["Sky is blue.", "Leaves are green.", "Roses are red."]
tokenize_sentences(sentences)

[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green', '.'],
 ['roses', 'are', 'red', '.']]




Use the two functions that you have just implemented to get the tokenized data.
- split the data into sentences
- tokenize those sentences

In [ ]:
def get_tokenized_data(data):
  sentences=split_into_sentences(data)
  tokenized_sentences=tokenize_sentences(sentences)
  return tokenized_sentences

In [ ]:
# test your function
x = "Sky is blue.\nLeaves are green\nRoses are red."
get_tokenized_data(x)

[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green'],
 ['roses', 'are', 'red', '.']]

### Split into train and test sets

Now run the cell below to split data into training and test sets.

In [ ]:
tokenized_data=get_tokenized_data(data)
random.seed(87)

train_size=int(len(tokenized_data)*0.8)
train_data=tokenized_data[0:train_size]
test_data=tokenized_data[train_size:]

In [ ]:
print("{} data are split into {} train and {} test set".format(
    len(tokenized_data), len(train_data), len(test_data)))

print("First training sample:")
print(train_data[0])
      
print("First test sample")
print(test_data[0])

47961 data are split into 38368 train and 9593 test set
First training sample:
['how', 'are', 'you', '?', 'btw', 'thanks', 'for', 'the', 'rt', '.', 'you', 'gon', 'na', 'be', 'in', 'dc', 'anytime', 'soon', '?', 'love', 'to', 'see', 'you', '.', 'been', 'way', ',', 'way', 'too', 'long', '.']
First test sample
['we', 'have', 'gone', 'pink', '!', 'join', 'team', 'germain', 'for', 'the', 'susan', 'g.', 'komen', 'race', 'for', 'the', 'cure', 'on', 'may', '15th', '!']


In [ ]:
def count_words(tokenized_sentences):
  word_counts={}

  for sentence in tokenized_sentences:
    for token in sentence:
      if token not in word_counts:
        word_counts[token]=1

      else:
        word_counts[token]+=1

  return word_counts

In [ ]:
# test your code
tokenized_sentences = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]
count_words(tokenized_sentences)

{'.': 3,
 'are': 2,
 'blue': 1,
 'green': 1,
 'is': 1,
 'leaves': 1,
 'red': 1,
 'roses': 1,
 'sky': 1}

### Handling 'Out of Vocabulary' words

If your model is performing autocomplete, but encounters a word that it never saw during training, it won't have an input word to help it determine the next word to suggest. The model will not be able to predict the next word because there are no counts for the current word. 
- This 'new' word is called an 'unknown word', or <b>out of vocabulary (OOV)</b> words.
- The percentage of unknown words in the test set is called the <b> OOV </b> rate. 

To handle unknown words during prediction, use a special token to represent all unknown words 'unk'. 

create a function that takes in a text document and a threshold 'count_threshold'.
- Any word whose count is greater than or equal to the threshold 'count_threshold' is kept in the closed vocabulary.
- used that you want to keep, returns the document containing only the word closed vocabulary and the word unk. 

In [ ]:
def get_words_with_nplus_frequency(tokenized_sentences,count_threshold):
  closed_vocab=[]

  word_counts=count_words(tokenized_sentences)

  for word,cnt in word_counts.items():
    if cnt>=count_threshold:
      closed_vocab.append(word)

  return closed_vocab



In [ ]:
# test your code
tokenized_sentences = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]
tmp_closed_vocab = get_words_with_nplus_frequency(tokenized_sentences, count_threshold=2)
print(f"Closed vocabulary:")
print(tmp_closed_vocab)

Closed vocabulary:
['.', 'are']


The words that appear 'count_threshold' times or more are in the 'closed vocabulary. 
- All other words are regarded as 'unknown'.
- Replace words not in the closed vocabulary with the token "<unk\>".

In [ ]:
def replace_oov_words_by_unk(tokenized_sentences,vocabulary,unknown_sentences='<unk>'):
  vocabulary=set(vocabulary)
  replaced_tokenized_sentences=[]

  for sentences in tokenized_sentences:
    replaced_sentence=[]
    for token in sentences:
      if token in vocabulary:
        replaced_sentence.append(token)
      else:
        replaced_sentence.append(unknown_sentences)
    replaced_tokenized_sentences.append(replaced_sentence)
  return replaced_tokenized_sentences

In [ ]:
tokenized_sentences = [["dogs", "run"], ["cats", "sleep"]]
vocabulary = ["dogs", "sleep"]
tmp_replaced_tokenized_sentences = replace_oov_words_by_unk(tokenized_sentences, vocabulary)
print(f"Original sentence:")
print(tokenized_sentences)
print(f"tokenized_sentences with less frequent words converted to '<unk>':")
print(tmp_replaced_tokenized_sentences)

Original sentence:
[['dogs', 'run'], ['cats', 'sleep']]
tokenized_sentences with less frequent words converted to '<unk>':
[['dogs', '<unk>'], ['<unk>', 'sleep']]


Now we are ready to process our data by combining the functions that we just implemented.

In [ ]:
def preprocess_data(train_data,test_data,count_threshold):
  vocabulary=get_words_with_nplus_frequency(train_data,count_threshold)
  train_data_replaced=replace_oov_words_by_unk(train_data,vocabulary)
  test_data_replaced=replace_oov_words_by_unk(test_data,vocabulary)

  return train_data_replaced,test_data_replaced,vocabulary

In [ ]:
# test your code
tmp_train = [['sky', 'is', 'blue', '.'],
     ['leaves', 'are', 'green']]
tmp_test = [['roses', 'are', 'red', '.']]

tmp_train_repl, tmp_test_repl, tmp_vocab = preprocess_data(tmp_train, 
                                                           tmp_test, 
                                                           count_threshold = 1)

print("tmp_train_repl")
print(tmp_train_repl)
print()
print("tmp_test_repl")
print(tmp_test_repl)
print()
print("tmp_vocab")
print(tmp_vocab)

tmp_train_repl
[['sky', 'is', 'blue', '.'], ['leaves', 'are', 'green']]

tmp_test_repl
[['<unk>', 'are', '<unk>', '.']]

tmp_vocab
['sky', 'is', 'blue', '.', 'leaves', 'are', 'green']


In this section, you will develop the n-grams language model.
- Assume the probability of the next word depends only on the previous n-gram.
- The previous n-gram is the series of the previous 'n' words.

In [ ]:
minimum_freq = 2
train_data_processed, test_data_processed, vocabulary = preprocess_data(train_data, 
                                                                        test_data, 
                                                                        minimum_freq)

In [ ]:
print("First preprocessed training sample:")
print(train_data_processed[0])
print()
print("First preprocessed test sample:")
print(test_data_processed[0])
print()
print("First 10 vocabulary:")
print(vocabulary[0:10])
print()
print("Size of vocabulary:", len(vocabulary))

First preprocessed training sample:
['how', 'are', 'you', '?', 'btw', 'thanks', 'for', 'the', 'rt', '.', 'you', 'gon', 'na', 'be', 'in', 'dc', 'anytime', 'soon', '?', 'love', 'to', 'see', 'you', '.', 'been', 'way', ',', 'way', 'too', 'long', '.']

First preprocessed test sample:
['we', 'have', 'gone', 'pink', '!', 'join', 'team', '<unk>', 'for', 'the', 'susan', 'g.', 'komen', 'race', 'for', 'the', 'cure', 'on', 'may', '15th', '!']

First 10 vocabulary:
['how', 'are', 'you', '?', 'btw', 'thanks', 'for', 'the', 'rt', '.']

Size of vocabulary: 14888


In [ ]:
def count_n_grams(data,n,start_token='<s>',end_token='<e>'):
  n_grams={}

  for sentence in data:
    sentence=[start_token]*(n)+ sentence +[end_token]
    sentence=tuple(sentence)

    m=len(sentence) if n==1 else len(sentence)-n+1

    for i in range(m):
      n_gram=sentence[i:i+n]
      if n_gram in n_grams.keys():
        n_grams[n_gram]+=1
      else:
        n_grams[n_gram]=1
  return n_grams


In [ ]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
print("Uni-gram:")
print(count_n_grams(sentences, 1))
print("Bi-gram:")
print(count_n_grams(sentences, 2))
print("Tri-gram:")
print(count_n_grams(sentences, 3))

Uni-gram:
{('<s>',): 2, ('i',): 1, ('like',): 2, ('a',): 2, ('cat',): 2, ('<e>',): 2, ('this',): 1, ('dog',): 1, ('is',): 1}
Bi-gram:
{('<s>', '<s>'): 2, ('<s>', 'i'): 1, ('i', 'like'): 1, ('like', 'a'): 2, ('a', 'cat'): 2, ('cat', '<e>'): 2, ('<s>', 'this'): 1, ('this', 'dog'): 1, ('dog', 'is'): 1, ('is', 'like'): 1}
Tri-gram:
{('<s>', '<s>', '<s>'): 2, ('<s>', '<s>', 'i'): 1, ('<s>', 'i', 'like'): 1, ('i', 'like', 'a'): 1, ('like', 'a', 'cat'): 2, ('a', 'cat', '<e>'): 2, ('<s>', '<s>', 'this'): 1, ('<s>', 'this', 'dog'): 1, ('this', 'dog', 'is'): 1, ('dog', 'is', 'like'): 1, ('is', 'like', 'a'): 1}




Next, estimate the probability of a word given the prior 'n' words using the n-gram counts.

$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_n)}{C(w_{t-1}\dots w_{t-n})} \tag{2} $$

This formula doesn't work when a count of an n-gram is zero..
- Suppose we encounter an n-gram that did not occur in the training data.  
- Then, the equation (2) cannot be evaluated (it becomes zero divided by zero).

A way to handle zero counts is to add k-smoothing.  
- K-smoothing adds a positive constant $k$ to each numerator and $k \times |V|$ in the denominator, where $|V|$ is the number of words in the vocabulary.

$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_n) + k}{C(w_{t-1}\dots w_{t-n}) + k|V|} \tag{3} $$


For n-grams that have a zero count, the equation (3) becomes $\frac{1}{|V|}$.
- This means that any n-gram with zero count has the same probability of $\frac{1}{|V|}$.

In [ ]:
def estimate_probability(word,previous_n_gram,
                         n_gram_counts,n_plus1_gram_counts,vocabulary_size,k=1.0):
  previous_n_gram=tuple(previous_n_gram)

  previous_n_gram_count=n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0

  denomitor= previous_n_gram_count+k*vocabulary_size

  n_plus1_gram=previous_n_gram+(word,)

  n_plus1_gram_counts=n_plus1_gram_counts[n_plus1_gram] if n_plus1_gram in n_plus1_gram_counts else 0

  numerator=n_plus1_gram_counts+k

  probability=numerator/denomitor

  return probability


In [ ]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)

bigram_counts = count_n_grams(sentences, 2)

tmp_prob = estimate_probability("cat", "a", unigram_counts, bigram_counts, len(unique_words), k=1)

print(f"The estimated probability of word 'cat' given the previous n-gram 'a' is: {tmp_prob:.4f}")

The estimated probability of word 'cat' given the previous n-gram 'a' is: 0.3333


In [ ]:

def word_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):
  previous_n_gram=tuple(previous_n_gram)
  vocabulary=vocabulary+['<e>','<unk>']
  vocabulary_size=len(vocabulary)

  probabilities={}

  for word in vocabulary:
    probability=estimate_probability(word,previous_n_gram,n_gram_counts,n_plus1_gram_counts,vocabulary_size,k=k)
    probabilities[word]=probability

  return probabilities



In [ ]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
word_probabilities("a", unigram_counts, bigram_counts, unique_words, k=1)

{'<e>': 0.09090909090909091,
 '<unk>': 0.09090909090909091,
 'a': 0.09090909090909091,
 'cat': 0.2727272727272727,
 'dog': 0.09090909090909091,
 'i': 0.09090909090909091,
 'is': 0.09090909090909091,
 'like': 0.09090909090909091,
 'this': 0.09090909090909091}

In [ ]:
# Additional test
trigram_counts = count_n_grams(sentences, 3)
word_probabilities(["<s>", "<s>"], bigram_counts, trigram_counts, unique_words, k=1)

{'<e>': 0.09090909090909091,
 '<unk>': 0.09090909090909091,
 'a': 0.09090909090909091,
 'cat': 0.09090909090909091,
 'dog': 0.09090909090909091,
 'i': 0.18181818181818182,
 'is': 0.09090909090909091,
 'like': 0.09090909090909091,
 'this': 0.18181818181818182}

### Count and probability matrices

As we have seen so far, the n-gram counts computed above are sufficient for computing the probabilities of the next word.  
- It can be more intuitive to present them as count or probability matrices.
- The functions defined in the next cells return count or probability matrices.

In [ ]:
def make_count_matrix(n_plus1_gram_counts,vocabulary):
  vocabulary = vocabulary + ["<e>", "<unk>"]

  n_grams=[]
  for n_plus1_gram in n_plus1_gram_counts.keys():
    n_gram=n_plus1_gram[0:-1]
    n_grams.append(n_gram)
  n_grams=list(set(n_grams))

  row_index={n_gram:i for i,n_gram in enumerate(n_grams)}
  col_index={word:j for j,word in enumerate(vocabulary)}

  nrow=len(n_grams)
  ncol=len(vocabulary)

  count_matrix=np.zeros((nrow,ncol))

  for n_plus1_gram,count in n_plus1_gram_counts.items():
    n_gram=n_plus1_gram[0:-1]
    word=n_plus1_gram[-1]

    if word not in vocabulary:
      continue
    i=row_index[n_gram]
    j=col_index[word]

    count_matrix[i,j]=count
  count_matrix=pd.DataFrame(count_matrix,index=n_grams,columns=vocabulary)
  return count_matrix


In [ ]:
sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
bigram_counts = count_n_grams(sentences, 2)

print('bigram counts')
display(make_count_matrix(bigram_counts, unique_words))

bigram counts


,like,this,a,dog,is,i,cat,<e>,<unk>
"(i,)",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(cat,)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
"(is,)",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(a,)",0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
"(dog,)",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"(like,)",0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
"(<s>,)",0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"(this,)",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Show trigram counts
print('\ntrigram counts')
trigram_counts = count_n_grams(sentences, 3)
display(make_count_matrix(trigram_counts, unique_words))


trigram counts


,like,this,a,dog,is,i,cat,<e>,<unk>
"(is, like)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"(like, a)",0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
"(<s>, this)",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
"(dog, is)",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(i, like)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"(<s>, i)",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(a, cat)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
"(<s>, <s>)",0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"(this, dog)",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


The following function calculates the probabilities of each word given the previous n-gram, and stores this in matrix form.

In [ ]:
def make_probability_matrix(n_plus1_gram_counts,vocabulary,k):
  count_matrix=make_count_matrix(n_plus1_gram_counts,vocabulary)
  count_matrix+=k
  prob_matrix=count_matrix.div(count_matrix.sum(axis=1),axis=0)
  return prob_matrix

In [ ]:
sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
bigram_counts = count_n_grams(sentences, 2)
print("bigram probabilities")
display(make_probability_matrix(bigram_counts, unique_words, k=1))

bigram probabilities


,like,this,a,dog,is,i,cat,<e>,<unk>
"(i,)",0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(cat,)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909
"(is,)",0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(a,)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909,0.090909
"(dog,)",0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000
"(like,)",0.090909,0.090909,0.272727,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
"(<s>,)",0.090909,0.181818,0.090909,0.090909,0.090909,0.181818,0.090909,0.090909,0.090909
"(this,)",0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000


In [ ]:
print("trigram probabilities")
trigram_counts = count_n_grams(sentences, 3)
display(make_probability_matrix(trigram_counts, unique_words, k=1))

trigram probabilities


,like,this,a,dog,is,i,cat,<e>,<unk>
"(is, like)",0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(like, a)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909,0.090909
"(<s>, this)",0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000
"(dog, is)",0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(i, like)",0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(<s>, i)",0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(a, cat)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909
"(<s>, <s>)",0.090909,0.181818,0.090909,0.090909,0.090909,0.181818,0.090909,0.090909,0.090909
"(this, dog)",0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000


## Part 4: Build an auto-complete system

In this section, you will combine the language models developed so far to implement an auto-complete system. 


In [ ]:
def suggest_a_word(previous_tokens,n_gram_counts,n_plus1_gram_counts,vocabulary,k=1.0,start_with=None):
  n=len(list(n_gram_counts.keys())[0])
  previous_n_gram=previous_tokens[-n:]

  probabilities=word_probabilities(previous_n_gram,n_gram_counts,n_plus1_gram_counts,vocabulary,k=k)

  suggestions=None

  max_prob=0

  for word,prob in probabilities.items():
    if start_with !=None:
      if not word.startswith(start_with):
        continue
    if prob>max_prob:
      suggestions=word

      max_prob=prob

  return suggestions,max_prob

In [ ]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)

previous_tokens = ["i", "like"]
tmp_suggest1 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)
print(f"The previous words are 'i like',\n\tand the suggested word is `{tmp_suggest1[0]}` with a probability of {tmp_suggest1[1]:.4f}")

print()
# test your code when setting the starts_with
tmp_starts_with = 'c'
tmp_suggest2 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0, start_with=tmp_starts_with)
print(f"The previous words are 'i like', the suggestion must start with `{tmp_starts_with}`\n\tand the suggested word is `{tmp_suggest2[0]}` with a probability of {tmp_suggest2[1]:.4f}")

The previous words are 'i like',
	and the suggested word is `a` with a probability of 0.2727

The previous words are 'i like', the suggestion must start with `c`
	and the suggested word is `cat` with a probability of 0.0909


In [ ]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):
    model_counts = len(n_gram_counts_list)
    suggestions = []
    for i in range(model_counts-1):
        n_gram_counts = n_gram_counts_list[i]
        n_plus1_gram_counts = n_gram_counts_list[i+1]
        
        suggestion = suggest_a_word(previous_tokens, n_gram_counts,
                                    n_plus1_gram_counts, vocabulary,
                                    k=k, start_with=start_with)
        suggestions.append(suggestion)
    return suggestions

In [ ]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
trigram_counts = count_n_grams(sentences, 3)
quadgram_counts = count_n_grams(sentences, 4)
qintgram_counts = count_n_grams(sentences, 5)

n_gram_counts_list = [unigram_counts, bigram_counts, trigram_counts, quadgram_counts, qintgram_counts]
previous_tokens = ["i", "like"]
tmp_suggest3 = get_suggestions(previous_tokens, n_gram_counts_list, unique_words, k=1.0)

print(f"The previous words are 'i like', the suggestions are:")
display(tmp_suggest3)

The previous words are 'i like', the suggestions are:


[('a', 0.2727272727272727),
 ('a', 0.2),
 ('like', 0.1111111111111111),
 ('like', 0.1111111111111111)]

In [ ]:
n_gram_counts_list = []
for n in range(1, 6):
    print("Computing n-gram counts with n =", n, "...")
    n_model_counts = count_n_grams(train_data_processed, n)
    n_gram_counts_list.append(n_model_counts)

Computing n-gram counts with n = 1 ...
Computing n-gram counts with n = 2 ...
Computing n-gram counts with n = 3 ...
Computing n-gram counts with n = 4 ...
Computing n-gram counts with n = 5 ...


In [ ]:
previous_tokens = ["i", "am", "to"]
tmp_suggest4 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest4)

The previous words are ['i', 'am', 'to'], the suggestions are:


[('be', 0.026981818181818183),
 ('go', 0.0001342552191716453),
 ('have', 0.00013430029546065002),
 ('how', 6.71591672263264e-05)]

In [ ]:
previous_tokens = ["i", "want", "to", "go"]
tmp_suggest5 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest5)

The previous words are ['i', 'want', 'to', 'go'], the suggestions are:


[('to', 0.014113404625714997),
 ('to', 0.004876441515650741),
 ('to', 0.0007373148334338763),
 ('home', 0.0004026035026504731)]

In [ ]:
previous_tokens = ["hey", "how", "are"]
tmp_suggest6 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest6)

The previous words are ['hey', 'how', 'are'], the suggestions are:


[('you', 0.02143263217737351),
 ('you', 0.0036105910671302486),
 ('how', 6.71591672263264e-05),
 ('how', 6.71591672263264e-05)]

In [ ]:
previous_tokens = ["hey", "how", "are", "you"]
tmp_suggest7 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest7)

The previous words are ['hey', 'how', 'are', 'you'], the suggestions are:


[("'re", 0.023333063274730618),
 ('?', 0.002882221931088694),
 ('?', 0.0017399451248076023),
 ('how', 6.71591672263264e-05)]

In [ ]:
previous_tokens = ["hey", "how", "are", "you"]
tmp_suggest8 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with="d")

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest8)

The previous words are ['hey', 'how', 'are', 'you'], the suggestions are:


[('do', 0.00907396905128413),
 ('doing', 0.001310100877767588),
 ('doing', 0.00033460483169376964),
 ('dc', 6.71591672263264e-05)]


## Part 3: Perplexity

In this section, you will generate the perplexity score to evaluate your model on the test set. 
- You will also use back-off when needed. 
- Perplexity is used as an evaluation metric of your language model. 
- To calculate the  the perplexity score of the test set on an n-gram model, use: 

$$ PP(W) =\sqrt[N]{ \prod_{t=n+1}^N \frac{1}{P(w_t | w_{t-n} \cdots w_{t-1})} } \tag{4}$$

- where $N$ is the length of the sentence.
- $n$ is the number of words in the n-gram (e.g. 2 for a bigram).
- In math, the numbering starts at one and not zero.

In code, array indexing starts at zero, so the code will use ranges for $t$ according to this formula:

$$ PP(W) =\sqrt[N]{ \prod_{t=n}^{N-1} \frac{1}{P(w_t | w_{t-n} \cdots w_{t-1})} } \tag{4.1}$$

The higher the probabilities are, the lower the perplexity will be. 
- The more the n-grams tell us about the sentence, the lower the perplexity score will be. 

In [ ]:
# calculate_perplexity
def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):

    # length of previous words
    n = len(list(n_gram_counts.keys())[0]) 
    
    # prepend <s> and append <e>
    sentence = ["<s>"] * n + sentence + ["<e>"]
    
    # Cast the sentence from a list to a tuple
    sentence = tuple(sentence)
    
    # length of sentence (after adding <s> and <e> tokens)
    N = len(sentence)
    
    product_pi = 1.0
    

    for t in range(n, N): 

        # get the n-gram preceding the word at position t
        n_gram = sentence[t-n:t]
        
        # get the word at position t
        word = sentence[t]
        
        # Estimate the probability of the word given the n-gram
        probability = estimate_probability(word,n_gram, n_gram_counts, n_plus1_gram_counts, len(unique_words), k=1)
        
        # Update the product of the probabilities
        product_pi *= 1 / probability

    # Take the Nth root of the product
    perplexity = product_pi**(1/float(N))
    
    ### END CODE HERE ### 
    return perplexity

In [ ]:
# test your code

sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)


perplexity_train1 = calculate_perplexity(sentences[0],
                                         unigram_counts, bigram_counts,
                                         len(unique_words), k=1.0)
print(f"Perplexity for first train sample: {perplexity_train1:.4f}")

test_sentence = ['i', 'like', 'a', 'dog']
perplexity_test = calculate_perplexity(test_sentence,
                                       unigram_counts, bigram_counts,
                                       len(unique_words), k=1.0)
print(f"Perplexity for test sample: {perplexity_test:.4f}")

Perplexity for first train sample: 2.8040
Perplexity for test sample: 3.9654
